# Preparación y Merge de Datos
## Customer Flight Activity & Loyalty History

## 1. Carga de librerías

En esta sección importamos las librerías necesarias para trabajar con los datos.

- **pandas** y **numpy** para manipulación de datos.
- **matplotlib** y **seaborn** para visualización posterior.
- Ajustamos algunas opciones de visualización para facilitar la lectura de tablas grandes.

Este paso es imprescindible antes de comenzar cualquier análisis.


In [1]:
# =========================================
# 1) Imports
# =========================================

"""
Librerías necesarias para:
- Manipulación de datos (pandas, numpy)
- Visualización (matplotlib, seaborn)
- Mostrar tablas de forma más clara en el notebook
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

# Configuración para que se vean mejor las tablas
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

print("✅ Librerías cargadas correctamente.")


✅ Librerías cargadas correctamente.


## 2. Carga de los datasets

Cargamos los dos archivos CSV proporcionados:

- **Customer Flight Activity**: contiene la actividad mensual de vuelos de cada cliente.
- **Customer Loyalty History**: contiene la información demográfica y de perfil de cada cliente.

En este punto verificamos que los datos se cargan correctamente y revisamos su tamaño.

In [3]:
# =========================================
# 2) Carga de datos
# =========================================

"""
Leemos los dos archivos CSV proporcionados en el ejercicio.

- Customer Flight Analysis.csv → actividad de vuelos mensual
- Customer Loyalty History.csv → perfil del cliente
"""

df_flight = pd.read_csv("../data/raw/Customer Flight Activity.csv")
df_loyalty = pd.read_csv("../data/raw/Customer Loyalty History.csv")

print("📊 Customer Flight Activity")
print(f"Filas: {df_flight.shape[0]} | Columnas: {df_flight.shape[1]}")

print("\n📊 Customer Loyalty History")
print(f"Filas: {df_loyalty.shape[0]} | Columnas: {df_loyalty.shape[1]}")

print("\n🔎 Vista rápida Flight:")
display(df_flight.head())

print("\n🔎 Vista rápida Loyalty:")
display(df_loyalty.head())



📊 Customer Flight Activity
Filas: 405624 | Columnas: 10

📊 Customer Loyalty History
Filas: 16737 | Columnas: 16

🔎 Vista rápida Flight:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0



🔎 Vista rápida Loyalty:


,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


Observamos que el dataset de vuelos contiene muchas más filas que el dataset de perfil.

Esto es coherente, ya que la tabla de vuelos registra actividad mensual (varias filas por cliente), mientras que la tabla de loyalty contiene una fila por cliente con información demográfica.


## 3. Validación de la clave de unión

Antes de realizar el merge, es fundamental validar la columna común entre ambos datasets: **Loyalty Number**.

Comprobamos:

- Si existen valores nulos en la clave.
- Si el número de clientes únicos coincide en ambos datasets.
- Si existen duplicados en la tabla de perfil.
- Si la tabla de vuelos tiene más de una fila por cliente-año-mes.

Este paso evita errores como duplicaciones no deseadas o pérdida de información en la unión.

In [4]:
# =========================================
# 3) EDA pre-merge #1 — Clave y granularidad
# =========================================

"""
Objetivo:
Comprobar si podemos hacer el merge sin problemas:

1) Nulos en la clave 'Loyalty Number'
2) Nº de clientes únicos en cada dataset
3) Duplicados de 'Loyalty Number' en el dataset de perfil (debería ser 0)
4) Duplicados de (Loyalty Number, Year, Month) en el dataset de vuelos (esperado: 0 si es 1 fila por cliente/mes)
"""

clave = "Loyalty Number"

print("🔑 Nulos en la clave")
print("Flight:", df_flight[clave].isna().sum())
print("Loyalty:", df_loyalty[clave].isna().sum())

print("\n👤 Clientes únicos (Loyalty Number)")
print("Flight:", df_flight[clave].nunique())
print("Loyalty:", df_loyalty[clave].nunique())

print("\n📌 Duplicados de Loyalty Number en Loyalty (perfil)")
print(df_loyalty.duplicated(subset=[clave]).sum())

print("\n📌 Duplicados de (Loyalty Number, Year, Month) en Flight (actividad)")
print(df_flight.duplicated(subset=[clave, "Year", "Month"]).sum())


🔑 Nulos en la clave
Flight: 0
Loyalty: 0

👤 Clientes únicos (Loyalty Number)
Flight: 16737
Loyalty: 16737

📌 Duplicados de Loyalty Number en Loyalty (perfil)
0

📌 Duplicados de (Loyalty Number, Year, Month) en Flight (actividad)
3936


Los resultados muestran que:

- No hay valores nulos en la clave.
- Ambos datasets tienen el mismo número de clientes únicos.
- La tabla de perfil tiene una única fila por cliente.
- La tabla de vuelos presenta duplicados por cliente-año-mes.

El merge podrá realizarse correctamente, pero antes debemos analizar esos duplicados para evitar doble conteo.


## 4. Inspección de duplicados en la tabla de vuelos

Detectamos duplicados por (Loyalty Number, Year, Month).

En esta sección analizamos:

- Cuántas filas están implicadas.
- Cuántos grupos cliente-mes están afectados.
- Si los registros duplicados son copias exactas o contienen información diferente.

Este análisis nos permitirá decidir cómo tratarlos correctamente.

In [5]:
# =========================================
# 4) Inspección de duplicados en Flight por cliente-año-mes
# =========================================

"""
Objetivo:
Entender qué está pasando con los duplicados de (Loyalty Number, Year, Month).
Vamos a:
1) Extraer solo las filas duplicadas (todos los registros implicados)
2) Ver algunos ejemplos ordenados
3) Ver cuántas repeticiones hay por grupo
"""

# 1) Todas las filas que forman parte de un duplicado (no solo "la segunda")
duplicados_flight = df_flight[df_flight.duplicated(subset=[clave, "Year", "Month"], keep=False)].copy()

print("📌 Filas implicadas en duplicados (cliente-año-mes):", duplicados_flight.shape[0])
print("📌 Nº de grupos cliente-año-mes duplicados:", duplicados_flight.groupby([clave, "Year", "Month"]).ngroups)

print("\n🔎 Ejemplos (primeros 10 registros duplicados ordenados):")
display(
    duplicados_flight.sort_values([clave, "Year", "Month"]).head(10)
)

# 2) Tamaño de los duplicados por grupo (2, 3, 4...)
repeticiones = (
    duplicados_flight
    .groupby([clave, "Year", "Month"])
    .size()
    .value_counts()
    .sort_index()
)

print("\n📊 Distribución de repeticiones por grupo (cuántos grupos tienen 2 filas, 3 filas, etc.):")
display(repeticiones.to_frame(name="n_grupos"))


📌 Filas implicadas en duplicados (cliente-año-mes): 7848
📌 Nº de grupos cliente-año-mes duplicados: 3912

🔎 Ejemplos (primeros 10 registros duplicados ordenados):


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
16942,101902,2017,2,0,0,0,0,0.0,0,0
16943,101902,2017,2,0,0,0,0,0.0,0,0
33843,101902,2017,3,0,0,0,0,0.0,0,0
33844,101902,2017,3,0,0,0,0,0.0,0,0
50744,101902,2017,4,4,0,4,1460,146.0,0,0
185796,101902,2017,4,4,4,8,2384,238.0,488,40
67645,101902,2017,5,9,3,12,2748,274.0,0,0
67646,101902,2017,5,7,0,7,3318,331.0,0,0



📊 Distribución de repeticiones por grupo (cuántos grupos tienen 2 filas, 3 filas, etc.):


,n_grupos
2,3888
3,24


Observamos que algunos duplicados son copias exactas, pero otros contienen valores distintos dentro del mismo cliente y mes.

Esto significa que no podemos eliminar todos los duplicados directamente sin perder información. Será necesario diferenciarlos antes de actuar.

## 5. Identificación de duplicados exactos y no exactos

En este paso distinguimos entre:

- Duplicados exactos: filas completamente iguales.
- Duplicados no exactos: misma clave cliente-mes pero con valores distintos.

Esta diferenciación es clave para decidir qué registros pueden eliminarse y cuáles deben consolidarse.

In [6]:
# =========================================
# 5) Duplicados exactos vs no exactos en Flight
# =========================================

"""
Objetivo:
Distinguir entre:
- Duplicados exactos: filas 100% iguales → se pueden eliminar.
- Duplicados no exactos: mismo cliente-año-mes pero valores distintos → hay que agregarlos.
"""

# 1) Duplicados exactos (todas las columnas iguales)
duplicados_exactos = df_flight.duplicated(keep="first").sum()
print("📌 Nº de filas duplicadas exactas (todas las columnas iguales):", duplicados_exactos)

# 2) Duplicados por cliente-año-mes
duplicados_cliente_mes = df_flight.duplicated(subset=[clave, "Year", "Month"]).sum()
print("📌 Nº de duplicados por (Loyalty Number, Year, Month):", duplicados_cliente_mes)

# 3) Dentro de las filas duplicadas por cliente-mes, ver cuántos grupos son exactos vs no exactos
dups = df_flight[df_flight.duplicated(subset=[clave, "Year", "Month"], keep=False)].copy()

# Contamos cuántas filas únicas reales hay dentro de cada grupo (cliente-año-mes)
# Si en un grupo hay 2 filas pero solo 1 fila única → eran exactos
unicos_por_grupo = dups.groupby([clave, "Year", "Month"]).apply(lambda x: x.drop_duplicates().shape[0])

print("\n📊 Nº de grupos duplicados que en realidad son copias exactas (solo 1 fila única en el grupo):")
print((unicos_por_grupo == 1).sum())

print("📊 Nº de grupos duplicados con información distinta (más de 1 fila única en el grupo):")
print((unicos_por_grupo > 1).sum())


📌 Nº de filas duplicadas exactas (todas las columnas iguales): 1864
📌 Nº de duplicados por (Loyalty Number, Year, Month): 3936

📊 Nº de grupos duplicados que en realidad son copias exactas (solo 1 fila única en el grupo):
1846
📊 Nº de grupos duplicados con información distinta (más de 1 fila única en el grupo):
2066


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14040\219657994.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unicos_por_grupo = dups.groupby([clave, "Year", "Month"]).apply(lambda x: x.drop_duplicates().shape[0])


Se identifican dos situaciones:

- Existen filas duplicadas exactas que pueden eliminarse sin pérdida de información.
- Existen registros con diferencias reales dentro del mismo cliente y mes, que deberán consolidarse para evitar doble conteo.

El siguiente paso será eliminar únicamente los duplicados exactos.

## 6. Eliminación de duplicados exactos

Eliminamos únicamente las filas que son completamente iguales en todas sus columnas.

De esta manera reducimos el ruido del dataset sin afectar a los registros que contienen información distinta.

In [7]:
# =========================================
# 6) Flight: eliminar duplicados exactos
# =========================================

"""
Objetivo:
Eliminar filas duplicadas exactas (todas las columnas iguales).
Esto NO afecta a los casos donde un mismo cliente-mes tiene información distinta.
"""

filas_antes = df_flight.shape[0]

df_flight_sin_exactos = df_flight.drop_duplicates().copy()

filas_despues = df_flight_sin_exactos.shape[0]

print("📉 Eliminación de duplicados exactos en Flight")
print("Filas antes:", filas_antes)
print("Filas después:", filas_despues)
print("Filas eliminadas:", filas_antes - filas_despues)

print("\n📌 Duplicados por (cliente, año, mes) tras quitar exactos:")
print(df_flight_sin_exactos.duplicated(subset=["Loyalty Number", "Year", "Month"]).sum())


📉 Eliminación de duplicados exactos en Flight
Filas antes: 405624
Filas después: 403760
Filas eliminadas: 1864

📌 Duplicados por (cliente, año, mes) tras quitar exactos:
2072


Se han eliminado los duplicados exactos correctamente.

Sin embargo, todavía existen duplicados por cliente-año-mes con información distinta, por lo que el siguiente paso será consolidar la tabla a una fila por cliente y mes.

## 7. Consolidación de la tabla de vuelos

Para evitar doble conteo, agrupamos la tabla por:

- Loyalty Number
- Year
- Month

Aplicamos:
- SUM en las métricas mensuales (vuelos, distancia, puntos).
- MAX en Total Flights, ya que puede representar un acumulado.

El objetivo es dejar una única fila por cliente y mes.

In [8]:
# =========================================
# 7) Flight: consolidar a nivel cliente-año-mes
# =========================================

"""
Objetivo:
Dejar Flight con 1 fila por (Loyalty Number, Year, Month), evitando doble conteo.

Regla de agregación:
- SUMA (porque son métricas del mes): 
  Flights Booked, Flights with Companions, Distance, Points Accumulated,
  Points Redeemed, Dollar Cost Points Redeemed

- MAX para Total Flights:
  Por el enunciado, puede ser un acumulado (incluye meses anteriores), así que
  sumar podría inflar el dato.
"""

filas_antes = df_flight_sin_exactos.shape[0]

df_flight_mensual = (
    df_flight_sin_exactos
    .groupby(["Loyalty Number", "Year", "Month"], as_index=False)
    .agg({
        "Flights Booked": "sum",
        "Flights with Companions": "sum",
        "Total Flights": "max",
        "Distance": "sum",
        "Points Accumulated": "sum",
        "Points Redeemed": "sum",
        "Dollar Cost Points Redeemed": "sum"
    })
)

filas_despues = df_flight_mensual.shape[0]

print("📌 Consolidación Flight a nivel cliente-año-mes")
print("Filas antes:", filas_antes)
print("Filas después:", filas_despues)
print("Filas reducidas:", filas_antes - filas_despues)

print("\n✅ Duplicados por (Loyalty Number, Year, Month) tras consolidar:")
print(df_flight_mensual.duplicated(subset=["Loyalty Number", "Year", "Month"]).sum())

print("\n🔎 Vista rápida del Flight consolidado:")
display(df_flight_mensual.head())


📌 Consolidación Flight a nivel cliente-año-mes
Filas antes: 403760
Filas después: 401688
Filas reducidas: 2072

✅ Duplicados por (Loyalty Number, Year, Month) tras consolidar:
0

🔎 Vista rápida del Flight consolidado:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100018,2017,2,2,2,4,1320,132.0,0,0
2,100018,2017,3,14,3,17,2533,253.0,438,36
3,100018,2017,4,4,0,4,924,92.0,0,0
4,100018,2017,5,0,0,0,0,0.0,0,0


Tras la consolidación, la tabla queda sin duplicados por cliente-año-mes.

La granularidad ahora es correcta y el dataset de vuelos está preparado para realizar el merge sin riesgo de inflar métricas.

## 8. Revisión de tipos y rangos

Antes del merge, revisamos:

- Los tipos de datos de ambas tablas.
- El rango de Year y Month.
- Un resumen básico de la variable Salary.

Este paso permite detectar posibles incoherencias antes de unir los datasets.

In [9]:
# =========================================
# 8) Revisión rápida de tipos (dtypes) y rangos básicos
# =========================================

"""
Objetivo:
Comprobar que las columnas clave para el análisis tienen tipos razonables
y que Year/Month tienen rangos coherentes.
"""

print("🧾 Tipos de datos — Flight mensual")
display(df_flight_mensual.dtypes.to_frame(name="dtype"))

print("\n🧾 Tipos de datos — Loyalty")
display(df_loyalty.dtypes.to_frame(name="dtype"))

print("\n📅 Rango de fechas en Flight (Year y Month)")
print("Años:", df_flight_mensual["Year"].min(), "→", df_flight_mensual["Year"].max())
print("Meses:", df_flight_mensual["Month"].min(), "→", df_flight_mensual["Month"].max())

print("\n💰 Salary (Loyalty) — vistazo rápido")
print(df_loyalty["Salary"].describe())


🧾 Tipos de datos — Flight mensual


,dtype
Loyalty Number,int64
Year,int64
Month,int64
Flights Booked,int64
Flights with Companions,int64
Total Flights,int64
Distance,int64
Points Accumulated,float64
Points Redeemed,int64
Dollar Cost Points Redeemed,int64



🧾 Tipos de datos — Loyalty


,dtype
Loyalty Number,int64
Country,object
Province,object
City,object
Postal Code,object
Gender,object
Education,object
Salary,float64
Marital Status,object
Loyalty Card,object



📅 Rango de fechas en Flight (Year y Month)
Años: 2017 → 2018
Meses: 1 → 12

💰 Salary (Loyalty) — vistazo rápido
count     12499.000000
mean      79245.609409
std       35008.297285
min      -58486.000000
25%       59246.500000
50%       73455.000000
75%       88517.500000
max      407228.000000
Name: Salary, dtype: float64


Los tipos de datos y los rangos de fechas son coherentes.

En Salary se observan valores nulos y algunos valores atípicos (por ejemplo negativos), pero en esta fase no realizamos limpieza profunda. La limpieza completa se realizará en el notebook de EDA posterior.

## 9. Merge de los datasets y guardado

Realizamos un LEFT JOIN desde la tabla de vuelos consolidada hacia la tabla de perfil.

Elegimos LEFT JOIN para:

- Mantener toda la actividad de vuelos.
- Añadir información demográfica y de perfil a cada observación mensual.

Finalmente guardamos el dataset unido para trabajar el análisis completo en otro notebook.

In [10]:
# =========================================
# 9) Merge + guardado del dataset unido
# =========================================

"""
Objetivo:
Unir el dataset mensual de actividad (df_flight_mensual) con el perfil del cliente (df_loyalty)
usando 'Loyalty Number' como clave.

Estrategia:
- LEFT JOIN desde Flight mensual: mantenemos toda la actividad de vuelos
- Añadimos las variables del perfil desde Loyalty History

Después:
- Guardamos el resultado a CSV para trabajar la limpieza y EDA completa en otro notebook.
"""

df_merged = df_flight_mensual.merge(df_loyalty, on="Loyalty Number", how="left")

print("✅ Merge completado")
print("Filas:", df_merged.shape[0], "| Columnas:", df_merged.shape[1])

# Comprobación mínima: nulos en la clave tras el merge (debería ser 0)
print("\n🔑 Nulos en Loyalty Number tras merge:", df_merged["Loyalty Number"].isna().sum())

# Guardado del CSV unido
ruta_salida = "../data/processed/dataset_unido.csv"
df_merged.to_csv(ruta_salida, index=False)

print(f"\n💾 Archivo guardado en: {ruta_salida}")

print("\n🔎 Vista rápida del dataset unido:")
display(df_merged.head())



✅ Merge completado
Filas: 401688 | Columnas: 25

🔑 Nulos en Loyalty Number tras merge: 0

💾 Archivo guardado en: ../data/processed/dataset_unido.csv

🔎 Vista rápida del dataset unido:


,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,100018,2017,1,3,0,3,1521,152.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
1,100018,2017,2,2,2,4,1320,132.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
2,100018,2017,3,14,3,17,2533,253.0,438,36,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
3,100018,2017,4,4,0,4,924,92.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN
4,100018,2017,5,0,0,0,0,0.0,0,0,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN


El merge se ha realizado correctamente y el número de filas se mantiene.

El dataset resultante combina la actividad mensual con las variables de perfil del cliente. Este archivo será la base para el análisis estadístico y las visualizaciones en la siguiente fase.